# Prepare

In [1]:
# general modules
import pandas as pd
import sqlite3 
import os
import pandas_profiling
import seaborn as sns
import sqlite3 as sql
import hashlib

In [2]:
# variables
INPUT_FOLDER = "input"
OUTPUT_FOLDER = "output"
DATA_FOLDER = "data"

xl_name = "global-superstore.xls" 
db_name = "superstore.db"

randon_state = 42

# Import

In [3]:
df_orders = pd.read_excel(os.path.join("..", INPUT_FOLDER, xl_name), sheet_name="Orders")
df_returns = pd.read_excel(os.path.join("..", INPUT_FOLDER, xl_name), sheet_name="Returns")
df_people = pd.read_excel(os.path.join("..", INPUT_FOLDER, xl_name), sheet_name="People")

### Column Inspection

In [4]:
df_orders.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority'],
      dtype='object')

In [5]:
df_returns.columns

Index(['Returned', 'Order ID', 'Market'], dtype='object')

In [6]:
df_people.columns

Index(['Person', 'Region'], dtype='object')

In [7]:
df_people

,Person,Region
0,Anna Andreadi,Central
1,Chuck Magee,South
2,Kelly Williams,East
3,Matt Collister,West
4,Deborah Brumfield,Africa
5,Larry Hughes,AMEA
6,Nicole Hansen,Canada
7,Giulietta Dortch,Caribbean
8,Nora Preis,Central Asia
9,Jack Lebron,North


In [8]:
# seems to be not relevant at all

### Integration

In [9]:
raw_df = df_orders.merge(
    df_returns[["Order ID", "Returned"]],
    on="Order ID",
    how="left",
)

In [10]:
raw_df.columns


Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country',
       'Postal Code', 'Market', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority', 'Returned'],
      dtype='object')

In [11]:
# replace NaN values in Returned with "No"
raw_df["Returned"].fillna("No", inplace=True)

In [12]:
# remove obviously useless columns
raw_df.drop(columns="Row ID", inplace=True)
raw_df.drop(columns="Order ID", inplace=True)

In [13]:
# anonymize customer name
raw_df["Customer Name"] = raw_df.apply(lambda x: hashlib.md5(x["Customer Name"].encode()).hexdigest(), axis=1)

# Store

In [14]:
conn = sql.connect(os.path.join("..", DATA_FOLDER, db_name))
raw_df.to_sql('superstore', conn, if_exists='replace')

C:\Users\Marc\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(
